In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed

import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from statsmodels.stats.multitest import multipletests

pybedtools.cleanup(remove_all=True)

In [2]:
cluster_type = 'SubType'
cluster = 'IG-CA2_Peak1'
output_dir = 'IG-CA2_Peak1'

or_cutoff = 1.3
neg_lgp_cutoff = 4
cpu = 40

## DMR hits

In [3]:
total_dmr_annot = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/SubDMRAnnotation/DMRAnnotation.h5ad'
)

In [5]:
hypo_dmr_bed_df = pd.read_csv(
    f'/home/hanliu/project/mouse_rostral_brain/DMR/SubType/HypoBed/{cluster}.HypoDMR.DMS2.bed',
    header=None,
    sep='\t',
    index_col=-1,
    names=['chrom', 'start', 'end', 'SubDMR'])

hyper_dmr_bed_df = pd.read_csv(
    f'/home/hanliu/project/mouse_rostral_brain/DMR/SubType/HyperBed/{cluster}.HyperDMR.DMS2.bed',
    header=None,
    sep='\t',
    index_col=-1,
    names=['chrom', 'start', 'end', 'SubDMR'])

## Background Hits

In [6]:
total_background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/MotifScan/BackgroundAdultTissueDMR.MotifHits.with_region_bed.h5ad'
)

## Motif hits contingency table

In [7]:
def run_enrichment(dmr_type, dmr_annot, background_annot):
    # calculate motif occurence, not considering hits here
    pos = (dmr_annot[:, motif_ids].X > 0).sum(axis=0)
    pos_total = dmr_annot.shape[0]

    neg = (background_annot.X > 0).sum(axis=0)
    neg_total = background_annot.shape[0]

    # contingency tables
    tables = {}
    for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
        table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
        tables[motif] = table

    # calculate motif enrichment results
    results = {}
    with ProcessPoolExecutor(cpu) as executor:
        fs = {}
        for motif, t in tables.items():
            f = executor.submit(stats.fisher_exact, t, alternative='greater')
            fs[f] = motif

        for f in as_completed(fs):
            motif = fs[f]
            odds, p = f.result()
            results[motif] = {'oddsratio': odds, 'p_value': p}
    motif_enrich_df = pd.DataFrame(results).T
    _, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
    motif_enrich_df['adj_p'] = p
    motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
        -np.inf, -300)

    # add counts info to results
    records = {}
    for motif, t in tables.items():
        tp, tn = t[0]
        fp, fn = t[1]
        tp_rate = tp / pos_total
        fp_rate = fp / neg_total
        records[motif] = dict(tp=tp,
                              tn=tn,
                              fp=fp,
                              fn=fn,
                              tp_rate=tp_rate,
                              fp_rate=fp_rate)
    counts = pd.DataFrame(records).T
    motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

    # save results
    motif_enrich_df = motif_enrich_df[motif_enrich_df['oddsratio'] > 1].copy()
    motif_enrich_df[cluster_type] = cluster
    motif_enrich_df['DMRType'] = dmr_type
    motif_enrich_df.to_msgpack(
        f'{output_dir}/{cluster}.{dmr_type}.motif_enrichment.msg')
    return motif_enrich_df

## Hypo DMR Enrichment

### Remove Overlap with BG

In [8]:
motif_ids = total_dmr_annot.var_names[total_dmr_annot.var['FeatureType'] == 'MotifHits']

dmr_bed = pybedtools.BedTool().from_dataframe(hypo_dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    total_background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = total_background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids]
background_motif_hits

View of AnnData object with n_obs × n_vars = 281229 × 719 
    obs: 'chrom', 'start', 'end'

In [13]:
dmr_annot = total_dmr_annot[hypo_dmr_bed_df.index, :]

motif_enrich_df = run_enrichment(dmr_type='Hypo',
                                 dmr_annot=dmr_annot,
                                 background_annot=background_motif_hits)

# print final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.


45

In [ ]:
# dmr_annot = total_dmr_annot[hyper_dmr_bed_df.index, :]
# motif_ids = dmr_annot.var_names[dmr_annot.var['FeatureType'] == 'MotifHits']
# motif_enrich_df = run_enrichment(dmr_type='Hyper',
#                                  dmr_annot=dmr_annot,
#                                  background_annot=background_motif_hits)
#
# # print final filter
# filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
#                                     &
#                                     (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
# filtered_motif_df.shape[0]